In [ ]:
import pandas as pd
import pytz
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import scipy.optimize as optimize
import math
import locale
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
from time import sleep
from datetime import datetime,timedelta,date,time
from tqdm import tqdm, tqdm_notebook
from dateutil.relativedelta import relativedelta
from pandas import NaT
sns.set();

#locale.setlocale( locale.LC_ALL, 'en_US' )

tqdm_notebook().pandas()

#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../creds.json"

#### Todo: If MSRB and reference data v1 provide the same field, such as coupon rate, then we should query both and raise a warning if they disagree. This type of checking can be kept clearly separate from the actual calculation code. For some MSRB fields like msrb.dated_date, we want to know how they map to reference data v1 fields such as issue_date.

In [19]:
def get_data():
    query = """
SELECT
  IFNULL(msrb.settlement_date,
    msrb.assumed_settlement_date) AS settlement_date,
  msrb.trade_date,
  msrb.cusip,
  msrb.dated_date,
  msrb.dollar_price,
  ref_data_v1.issue_price,
  msrb.coupon AS coupon_rate,
  ref_data_v1.interest_payment_frequency,
  ref_data_v1.next_call_date,
  ref_data_v1.par_call_date,
  ref_data_v1.next_call_price,
  ref_data_v1.par_call_price,
  msrb.maturity_date AS maturity_date,
  ref_data_v1.previous_coupon_payment_date,
  ref_data_v1.next_coupon_payment_date,
  ref_data_v1.first_coupon_date,
  ref_data_v1.coupon_type,
  ref_data_v1.muni_security_type,
  ref_data_v1.called_redemption_type,
  ref_data_v1.refund_date,
  ref_data_v1.refund_price,
  ref_data_v1.is_callable,
  ref_data_v1.is_called,
  ref_data_v1.call_timing,
  msrb.yield,
  ref_data_v1.basic_assumed_maturity_date,
  ref_data_v1.maturity_date,
  ref_data_v1.called_redemption_date
FROM
  `eng-reactor-287421.MSRB.msrb_all` AS msrb
INNER JOIN
  `eng-reactor-287421.reference_data_v1.reference_data_flat` AS ref_data_v1
ON
  msrb.cusip = ref_data_v1.CUSIP
WHERE
  msrb.trade_date = '2021-01-13'
  AND '2021-01-13' BETWEEN ref_data_v1.ref_valid_from_date
  AND ref_data_v1.ref_valid_to_date
  AND NOT ref_data_v1.default_indicator
  --AND ref_data_v1.callable IS FALSE
  --AND ref_data_v1.next_call_date is not null
  --AND msrb.yield > 0
  AND msrb.maturity_date > DATE_ADD(IFNULL(msrb.settlement_date,
      msrb.assumed_settlement_date),INTERVAL 6 month)
  AND ref_data_v1.next_call_date > DATE_ADD(IFNULL(msrb.settlement_date,
      msrb.assumed_settlement_date),INTERVAL 6 month)
  --AND (ref_data_v1.next_call_date IS NOT NULL AND 
  --  (ref_data_v1.called_redemption_type IS NULL OR
  --  ref_data_v1.called_redemption_type NOT IN (1,2,3,4,5,6,13,14,15,17)))
    """
         
    dataframe = (bqclient.query(query).result().to_dataframe())
    return dataframe

In [ ]:
bqclient = bigquery.Client()
muni_df = get_data()

muni_security_type_dict = {0:'Unknown',	1:'Special assessment',	2:'Double barreled',	3:'Lease-rental',	4:'Fuel/vehicle tax',	5:'Unlimited G.O.',	6:'Limited G.O.',	7:'Other',	8:'Revenue',	9:'Sales/excise tax',	10:'Tax allocation/increment',	11:'U.S. government-backed',	12:'Tobacco state appropriated',	13:'Tobacco settlement non-appropriated',	14:'Federal grant',	15:'Cigarette tax',	16:'Hotel tax',	17:'Miscellaneous tax',	18:'Personal income tax',	19:'Pilot'}
muni_df['muni_security_type']= muni_df['muni_security_type'].map(muni_security_type_dict) 

coupon_type_dict = {0:'Unknown',	1:'Short term discount',	2:'Fixed rate - unconfirmed',	3:'Adjustable rate',	4:'Zero coupon',	5:'Floating rate',	6:'Index Linked',	7:'Stepped coupon',	8:'Fixed rate',	9:'Stripped convertible',	10:'Deferred interest',	11:'Floating rate @ floor',	12:'Stripped tax credit',	13:'Inverse floating',	14:'Stripped coupon principal',	15:'Linked inverse floater',	16:'Flexible rate',	17:'Original issue discount',	18:'Stripped principal',	19:'Reserve CUSIP',	20:'Variable rate',	21:'Stripped coupon',	22:'Floating auction rate',	23:'Tax credit',	24:'Tax credit OID',	25:'Stripped coupon payment',	26:'Stepped up stepped down',	27:'Credit Sensitive',	28:'Pay in kind',	29:'Range',	30:'Digital',	31:'Reset'}
muni_df['coupon_type']= muni_df['coupon_type'].map(coupon_type_dict) 

#frequency_dict_text = {0:'Unknown',	1:'Semiannually',	2:'Monthly',	3:'Annually',	4:'Weekly',	5:'Quarterly',	6:'Every 2 years',	7:'Every 3 years',	8:'Every 4 years',	9:'Every 5 years',	10:'Every 7 years',	11:'Every 8 years',	12:'Biweekly',	13:'Changeable',	14:'Daily',	15:'Term mode',	16:'Interest at maturity',	17:'Bimonthly',	18:'Every 13 weeks',	19:'Irregular',	20:'Every 28 days',	21:'Every 35 days',	22:'Every 26 weeks',	23:'Not Applicable',	24:'Tied to prime',	25:'One time',	26:'Every 10 years',	27:'Frequency to be determined',	28:'Mandatory put',	29:'Every 52 weeks',	30:'When interest adjusts-commercial paper',	31:'Zero coupon',	32:'Certain years only',	33:'Under certain circumstances',	34:'Every 15 years',	35:'Custom',	36:'Single Interest Payment'}
frequency_dict = {0:None,1:2,2:12,3:1,4:52,5:4,6:0.5,7:0.33333,8:0.25,9:0.2,10:1/7,11:1/8,13:44,14:360,16:0,23:None}
muni_df['interest_payment_frequency']= muni_df['interest_payment_frequency'].map(frequency_dict) 

muni_df['coupon_rate'] = muni_df['coupon_rate'].astype(float)
muni_df['next_call_price'] = muni_df['next_call_price'].astype(float)
muni_df["settlement_date"] = pd.to_datetime(muni_df["settlement_date"])
muni_df["first_coupon_date"] = pd.to_datetime(muni_df["first_coupon_date"])
muni_df["previous_coupon_payment_date"] = pd.to_datetime(muni_df["previous_coupon_payment_date"])
muni_df["next_coupon_payment_date"] = pd.to_datetime(muni_df["next_coupon_payment_date"])
muni_df["maturity_date"] = pd.to_datetime(muni_df["maturity_date"])
muni_df["refund_date"] = pd.to_datetime(muni_df["refund_date"])

muni_df["coupon_type"].unique()
#print(muni_df["coupon_type"].value_counts())
print(muni_df["coupon_type"].value_counts().plot(kind = 'barh'))

#muni_df = muni_df[muni_df.apply(lambda x: x["coupon_type"] in ['Zero coupon',],axis=1)]
#muni_df = muni_df[muni_df.apply(lambda x: x["coupon_type"] in ['Fixed rate',],axis=1)]
#muni_df = muni_df[muni_df.apply(lambda x: x["coupon_type"] in ['Original issue discount',],axis=1)]
muni_df = muni_df[muni_df.apply(lambda x: x["coupon_type"] in ['Fixed rate', 'Original issue discount', 'Zero coupon',],axis=1)]

len(muni_df)
#muni_df#.to_csv('01132020_Fixed_rate_trades.csv')

In [26]:
#delete: visualize rates
#sns.distplot(muni_df["coupon_rate"]);
#fig = plt.gcf()

In [27]:
def diff_in_days(end_date,start_date, convention="360/30"):
    #See MSRB Rule 33-G for details
    Y2 = end_date.year
    Y1 = start_date.year
    M2 = end_date.month
    M1 = start_date.month
    D2 = end_date.day #(end_date - relativedelta(days=1)).day 
    D1 = start_date.day
    if convention == "360/30":
        D1 = min(D1, 30)
        if D1 == 30: D2 = min(D2,30)
        difference_in_days = (Y2 - Y1) * 360 + (M2 - M1) * 30 + (D2 - D1)
    else: 
        print("unknown convention", convention)
    return difference_in_days

def actual_diff_in_days(end_date,start_date):
    end_date = datetime(end_date.year, end_date.month, end_date.day)
    start_date = datetime(start_date.year, start_date.month, start_date.day)
    return (end_date - start_date).days #/daycount_convention

def get_next_coupon_date(first_coupon_date,start_date,time_delta):
    date = first_coupon_date
    while date < start_date:
        date = date + time_delta

    next_coupon_date = date
    return next_coupon_date

def get_previous_coupon_date(first_coupon_date,settlement_date,time_delta):
    date = first_coupon_date
    while date < settlement_date:
        date = date + time_delta

    prev_coupon_date = date - time_delta
    return prev_coupon_date

##### we doubt that get_time_delta_from_interest_frequency works with every value in the coupon_frequency dictionary. Add some checks for correctness, or at least a detailed comment.

In [13]:
def get_time_delta_from_interest_frequency(interest_payment_frequency):
    #raise an error if interest_payment_frequency does not divide into 12: 
    if interest_payment_frequency != 0: 
        #delta = 12/interest_payment_frequency 
        if interest_payment_frequency <= 12:
            delta = 12/interest_payment_frequency
            time_delta = relativedelta(months=delta)
        elif interest_payment_frequency > 12 and interest_payment_frequency <= 52:
            delta = 52/interest_payment_frequency
            time_delta = relativedelta(weeks=delta)
    else: 
        time_delta = 0
    return(time_delta)

In [14]:
def get_yield(cusip,prev_coupon_date,first_coupon_date,next_coupon_date,end_date,settlement_date,dated_date,
              interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,
              called_redemption_type,refund_date,guess):

    # Single Trade Debug: remove this last. 
    if cusip == '786285BD7' and dollar_price == 99.800:
        print([cusip,prev_coupon_date,first_coupon_date,next_coupon_date,end_date,settlement_date,dated_date,
             interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,
            called_redemption_type,refund_date,guess])
    
   #Y is the yield
    B = 360 # the number of days in the year (computed in accordance with the provisions of section (e) below);        
    P = dollar_price # the dollar price of the security for each $100 par value;
    R = coupon_rate # the annual interest rate expressed as a percentage, i.e., dollars per $100;
    RV = par # the redemption value of the security per $100 par value; and
    M = interest_payment_frequency 
    number_of_interest_payments = 0

    if R != 0 and interest_payment_frequency !=0: # Coupon paid every M periods
        last_date = next_coupon_date

        # count the coupon payment periods left: 
        while last_date <= end_date: # end_date: maturity_date or next_call_date
            number_of_interest_payments += 1
            last_date += time_delta

        final_coupon_date = last_date - time_delta

        if pd.isnull(prev_coupon_date): prev_coupon_date = dated_date

        N = number_of_interest_payments    
        D = diff_in_days(end_date,final_coupon_date,"360/30") #Time from final Payment to Redemption, can be Zero  
        A = diff_in_days(settlement_date,prev_coupon_date,"360/30") # accrued days from beginning of the interest payment period
        E = B/M # number of days in interest payment period 

        # formula to handle one coupon left paid at maturity (or call date)
        if end_date<=next_coupon_date:
            DTC = diff_in_days(end_date,prev_coupon_date) # accrual days for final paid coupon
            DA = diff_in_days(settlement_date,prev_coupon_date) # accrual days for coupon owed in dirty price
            DM = diff_in_days(end_date,settlement_date) # hold period days
            
            # formula is ((Redemption Value + Coupon Paid at Maturity)/Dirty Price - 1) * 
            # (100 *  Payments per Year)/(Hold Period scaled for payments per year)
            return round( (((RV+(R/M)*(DTC*M/B))/(P+(R/M)*(DA*M/B)))-1)*(M*100)/(DM*M/B),3)

        else:
            #print('P=%s\nRV=%s\nR=%s\nD=%s\nB=%s\nY=%s\nM=%s\nN=%s\nE=%s\nA=%s\nrange(N)==%s\n' %(P,RV,R,D,B,'_',M,N,E,A,range(N)))    
            ytm_func = lambda Y: (RV + R*D/B)/(1+Y/M)**(N-1 + (E-A)/E + D/E) + sum([(R/M)/(1+Y/M)**(K+(E-A)/E) for K in range(N)]) - (P + R*A/B)        
                
    elif R != 0 and interest_payment_frequency == 0:
        # Interest at maturity, Rule 33-G (b)(i)A(b) - why do we get them as negatives??
        A = diff_in_days(settlement_date,dated_date,"360/30")
        DIR = diff_in_days(end_date,dated_date,"360/30")
        ytm_func = lambda Y: ((RV + (DIR/B*R))/(1-((DIR-A)/B*Y)))-(P + R*A/B)
        return round(optimize.newton(ytm_func, guess,maxiter = 100)*100,3)*-1 #fix this * -1. (check by pluggin in the positive)
    
    elif R == 0:
        # for Zero coupon use semi annual 360/30 convention
        periods = diff_in_days(end_date,settlement_date)/180.0
        ytm = ((par/P)**(1/periods)-1)*200
        return round(ytm,3)
    try:
        return round(optimize.newton(ytm_func, guess,maxiter = 100)*100,3)
    except Exception as e:
        print(e)
        return None
    
#get_yield('645916TV9', pd.to_datetime('2020-10-01 00:00:00'), pd.to_datetime('2003-10-01 00:00:00'), pd.to_datetime('2021-04-01 00:00:00'), date(2021, 2, 6), pd.to_datetime('2021-01-15 00:00:00'), date(2003, 4, 9), 2.0, 108.115, 5.8, 110.0, relativedelta(months=+6), pd.to_datetime('2025-04-01 00:00:00'), True, None, NaT, 0.01)

In [15]:
def compute_yield(settlement_date,trade_date,cusip,dated_date,dollar_price,coupon_rate,interest_payment_frequency,next_call_date,
                  next_call_price,par_call_date,par_call_price,maturity_date,previous_coupon_payment_date,next_coupon_payment_date,first_coupon_date,
                  coupon_type,muni_security_type,called_redemption_type,refund_date,refund_price,is_callable,call_timing,guess):

    time_delta = get_time_delta_from_interest_frequency(interest_payment_frequency)
    my_prev_coupon_date,my_next_coupon_date = None, None
    # Explain: why is data missing? Why do we need to change an exisiting next_coupon_payment_date?   
    if coupon_rate != 0 and interest_payment_frequency != 0:
        my_next_coupon_date = pd.to_datetime(next_coupon_payment_date)
        my_prev_coupon_date = pd.to_datetime(previous_coupon_payment_date)

        if (pd.isnull(previous_coupon_payment_date)):
            my_prev_coupon_date = get_previous_coupon_date(first_coupon_date,settlement_date,time_delta)
        if  (pd.isnull(next_coupon_payment_date)):
            my_next_coupon_date = get_next_coupon_date(first_coupon_date,settlement_date,time_delta)
        if next_coupon_payment_date < settlement_date and pd.isnull(previous_coupon_payment_date):
            my_next_coupon_date = get_next_coupon_date(first_coupon_date,settlement_date,time_delta)
     
    if not pd.isnull(called_redemption_type) and called_redemption_type != 19 and called_redemption_type != 18: # and coupon_rate !=0: 
        # Bond was Called:
        if called_redemption_type == 1 or called_redemption_type == 5:
            end_date = maturity_date
        else:
            end_date = refund_date              
        if not pd.isnull(refund_price):
            par = refund_price
        elif not pd.isnull(next_call_price): 
            par = next_call_price
        else: 
            par = 100
        ytm = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        return ytm
                    
    if pd.isnull(next_call_date) or pd.isnull(next_call_price):
        # Non callable or no call info:
        end_date = maturity_date
        par = 100
        ytm = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        return ytm
    
    else:
        # Callable:
        
        ytf = float("inf")
        ytm = float("inf")
        yta = float("inf")
        ytp = float("inf")        
        
        if not pd.isnull(par_call_date):
            end_date = par_call_date
            par = par_call_price    
            ytp = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)

        end_date = next_call_date #+ relativedelta(days=1)
        par = next_call_price    
        ytf = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        
        end_date = maturity_date
        par = 100
        ytm = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        
        '''
        # yield to anytime call below does not change anything: 
        if call_timing == 1 and trade_date >= next_call_date - relativedelta(days=30): 
            # calculate 30 days from today only if you are less than 30 days to the next call date: 
            end_date = trade_date + relativedelta(days=30)
            par = 100
            yta = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
            #print(next_call_date)
            #print('ytm: %s, ytf: %s, yta: %s' % (ytm, ytf,yta))
        '''
        yields = (ytf, ytm, yta,ytp)
    
        if cusip == '786285BD7' and dollar_price == 99.800: print('ytm: %s, ytf: %s, yta: %s, ytp: %s' % (ytm, ytf,'yta',ytp))
        return(min(yields))


In [16]:
def compute_assumed_maturity_date(settlement_date,trade_date,cusip,dated_date,dollar_price,coupon_rate,interest_payment_frequency,next_call_date,
                  next_call_price,par_call_date,par_call_price,maturity_date,previous_coupon_payment_date,next_coupon_payment_date,first_coupon_date,
                  coupon_type,muni_security_type,called_redemption_type,refund_date,refund_price,is_callable,call_timing,guess):

    time_delta = get_time_delta_from_interest_frequency(interest_payment_frequency)
    my_prev_coupon_date,my_next_coupon_date = None, None
    # Explain: why is data missing? Why do we need to change an exisiting next_coupon_payment_date?   
    if coupon_rate != 0 and interest_payment_frequency != 0:
        my_next_coupon_date = pd.to_datetime(next_coupon_payment_date)
        my_prev_coupon_date = pd.to_datetime(previous_coupon_payment_date)

        if (pd.isnull(previous_coupon_payment_date)):
            my_prev_coupon_date = get_previous_coupon_date(first_coupon_date,settlement_date,time_delta)
        if  (pd.isnull(next_coupon_payment_date)):
            my_next_coupon_date = get_next_coupon_date(first_coupon_date,settlement_date,time_delta)
        if next_coupon_payment_date < settlement_date and pd.isnull(previous_coupon_payment_date):
            my_next_coupon_date = get_next_coupon_date(first_coupon_date,settlement_date,time_delta)
     
    if not pd.isnull(called_redemption_type) and called_redemption_type != 19 and called_redemption_type != 18: # and coupon_rate !=0: 
        # Bond was Called:
        if called_redemption_type == 1 or called_redemption_type == 5:
            end_date = maturity_date
        else:
            end_date = refund_date              
        if not pd.isnull(refund_price):
            par = refund_price
        elif not pd.isnull(next_call_price): 
            par = next_call_price
        else: 
            par = 100
        ytm = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        return end_date
                    
    if pd.isnull(next_call_date) or pd.isnull(next_call_price):
        # Non callable or no call info:
        end_date = maturity_date
        par = 100
        ytm = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        return end_date
    
    else:
        # Callable:
        
        ytf = float("inf")
        ytm = float("inf")
        yta = float("inf")
        ytp = float("inf")        
        
        if not pd.isnull(par_call_date):
            end_date = par_call_date
            ytp_end_date = par_call_date
            par = par_call_price    
            ytp = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)

        end_date = next_call_date #+ relativedelta(days=1)
        ytf_end_date = next_call_date
        par = next_call_price    
        ytf = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        
        end_date = maturity_date
        ytm_end_date = maturity_date
        par = 100
        ytm = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
        
        '''
        # yield to anytime call below does not change anything: 
        if call_timing == 1 and trade_date >= next_call_date - relativedelta(days=30): 
            # calculate 30 days from today only if you are less than 30 days to the next call date: 
            end_date = trade_date + relativedelta(days=30)
            par = 100
            yta = get_yield(cusip,my_prev_coupon_date,first_coupon_date,my_next_coupon_date,end_date,settlement_date,dated_date,interest_payment_frequency,dollar_price,coupon_rate,par,time_delta,maturity_date,is_callable,called_redemption_type,refund_date,guess)
            #print(next_call_date)
            #print('ytm: %s, ytf: %s, yta: %s' % (ytm, ytf,yta))
        '''
        yields = (ytf, ytm, yta,ytp)
    
        if cusip == '786285BD7' and dollar_price == 99.800: print('ytm: %s, ytf: %s, yta: %s, ytp: %s' % (ytm, ytf,'yta',ytp))
        if min(yields) == ytf:
            return ytf_end_date
        elif min(yields) == ytm:
            return ytm_end_date
        else:
            return ytp_end_date


In [64]:
def advanced_assumed_maturity_date(price,is_called,is_callable,called_redemption_date,next_call_date,maturity_date,
                                    par_call_date,interest_payment_frequency):
    # if a bond has been called then it is retired on its refund date or maturity date
    if is_called:
        return called_redemption_date
    # callable bonds are the tricky case
    if is_callable:
        # first consider bonds trading at a discount
        if price <100:
            # assume zero coupons bonds have a YTW at next call date
            if interest_payment_frequency == 0:
                return next_call_date
            # otherwise, assume the YTW is at maturity
            return maturity_date
    
        # some bonds are first callable at a premium and then at par
        # blunt assumption: par_call_date is the YTW date
        if not par_call_date is None:
            if par_call_date!=next_call_date:
                return par_call_date
        
        # for callable bonds, if none of the other situations have obtained, then assume YTW at next call date
        return next_call_date
    
    # if a bond is neither called nor callable, it matures at its maturity date
    return maturity_date
    

In [ ]:
# Generate YTW
guess = 0.01

muni_df['computed_ytw'] = muni_df.progress_apply(lambda x: compute_yield(x.settlement_date,x.trade_date,x.cusip,x.dated_date,x.dollar_price,x.coupon_rate,x.interest_payment_frequency,x.next_call_date,x.next_call_price,x.par_call_date,x.par_call_price,x.maturity_date,x.previous_coupon_payment_date,x.next_coupon_payment_date,x.first_coupon_date,x.coupon_type,x.muni_security_type,	x.called_redemption_type,	x.refund_date,x.refund_price,	x.is_callable,	x.call_timing,guess),axis=1)

In [ ]:
# Generate assumed maturity date: 
guess = 0.01

muni_df['advanced_assumed_maturity_date'] = muni_df.progress_apply(lambda x: advanced_assumed_maturity_date(
    x.dollar_price,
    x.is_called,
    x.is_callable,
    x.called_redemption_date,
    x.next_call_date,
    x.maturity_date,
    x.par_call_date,
    x.interest_payment_frequency
),axis=1)

In [ ]:
# advanced rule of thumb: 
guess = 0.01

muni_df['computed_assumed_maturity_date'] = muni_df.progress_apply(lambda x: compute_assumed_maturity_date(x.settlement_date,x.trade_date,x.cusip,x.dated_date,x.dollar_price,x.coupon_rate,x.interest_payment_frequency,x.next_call_date,x.next_call_price,x.par_call_date,x.par_call_price,x.maturity_date,x.previous_coupon_payment_date,x.next_coupon_payment_date,x.first_coupon_date,x.coupon_type,x.muni_security_type,	x.called_redemption_type,	x.refund_date,x.refund_price,	x.is_callable,	x.call_timing,guess),axis=1)

In [ ]:
len(muni_df[muni_df.computed_assumed_maturity_date != muni_df.advanced_assumed_maturity_date])/len(muni_df)

In [ ]:
muni_df[muni_df.computed_assumed_maturity_date != muni_df.advanced_assumed_maturity_date] 

In [ ]:
plt.scatter(muni_df.advanced_assumed_maturity_date.tolist(), muni_df.computed_assumed_maturity_date.tolist(),c='DarkBlue')
plt.xlabel("ice flat assumed maturity")
plt.ylabel("ytw calculator assumed maturity")
plt.show()

In [ ]:
s = "'645916TV9', Timestamp('2020-10-01 00:00:00'), Timestamp('2003-10-01 00:00:00'), Timestamp('2021-04-01 00:00:00'), Timestamp('2025-04-01 00:00:00'), Timestamp('2021-01-15 00:00:00'), datetime.date(2003, 4, 9), 2.0, 108.115, 5.8, 100, relativedelta(months=+6), Timestamp('2025-04-01 00:00:00'), True, nan, NaT, 0.01"
s.replace('Timestamp','pd.to_datetime').replace('datetime.date','date').replace('nan','None')

In [ ]:
ax = muni_df.plot.scatter(x='yield',y='computed_ytw', c='DarkBlue')

In [163]:
muni_df['ytw_delta'] = muni_df['computed_ytw'] - muni_df['yield']

In [ ]:
delta = muni_df.sort_values(by=['ytw_delta'])#,ascending=False)
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 30
delta = delta[delta.ytw_delta != 0]
delta

In [ ]:
muni_df[muni_df.computed_ytw == muni_df['yield']]
#muni_df["computed_ytw"].corr(muni_df["yield"])

In [ ]:
#how many did we get right: 
print(len(muni_df[muni_df['computed_ytw'] == muni_df['yield']])/len(muni_df))
print(len(muni_df[abs(muni_df['computed_ytw'] - muni_df['yield']) <= 0.001])/len(muni_df))